In [1]:
# Basic imports
import os
import sys
import time
import pickle
from itertools import product
import warnings

# System path modification
sys.path.insert(0, '..')

# Data handling
import pandas as pd
import numpy as np

# Machine learning imports
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import (
    LinearRegression, Lasso, LassoCV, MultiTaskLasso, MultiTaskLassoCV,
    ElasticNet, ElasticNetCV, MultiTaskElasticNet, MultiTaskElasticNetCV
)
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

from sklearn.cross_decomposition import PLSRegression
from sklearn.inspection import permutation_importance

# Custom modules
from src.train import *
from src.functions import *
from src.plots import *
from src.dataset import *
from src.multixgboost import *
from src.wrapper import *
from src.debug import *

# Visualizatiokn 
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning and machine learning specific 
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
import xgboost as xgb
import shap

from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

from pytorch_tabular.models import (
    GatedAdditiveTreeEnsembleConfig,
    DANetConfig,
    TabTransformerConfig,
    FTTransformerConfig,
    TabNetModelConfig,
)

# Ignore warnings
warnings.filterwarnings("ignore")

# Print CUDA availability for PyTorch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

from omegaconf import DictConfig
torch.serialization.safe_globals([DictConfig])

True
1


## Load data 

In [2]:
data = load_pickle_data_palettes()

results_pickle_folder = "../pickle/"

# Unpack data
df_X, df_y, df_all, df_FinalCombination = data["df_X"], data["df_y"], data["df_all"], data["df_FinalCombination"]
dict_select = data["dict_select"]

# Unpack colormaps
full_palette, gender_palette, dx_palette = data["colormaps"].values()

# Train-Test Split

In [4]:
idx_train = list(df_X.isna().any(axis=1))
idx_test = list(~df_X.isna().any(axis=1))

set_intersect_rid = set(df_all[idx_train].RID).intersection(set(df_all[idx_test].RID))
intersect_rid_idx = df_all.RID.isin(set_intersect_rid)

for i, bool_test in enumerate(idx_test): 
    if intersect_rid_idx.iloc[i] & bool_test:
        idx_test[i] = False
        idx_train[i] = True
        
df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]] = df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]].astype("int", errors='ignore')

df_X_train = df_X.loc[idx_train]
df_X_test = df_X.loc[idx_test]

df_y_train = df_y.loc[idx_train]
df_y_test = df_y.loc[idx_test]

c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

In [ ]:
df_all.SubjectID.iloc[idx_test]

3609    128_S_2002
5631    116_S_4167
5662    033_S_4176
5780    098_S_4215
5950    018_S_4349
6069    941_S_4292
6077    116_S_4453
6085    135_S_4489
6224    033_S_4505
6400    014_S_4576
6429    073_S_4300
7021    003_S_2374
7192    033_S_4179
Name: SubjectID, dtype: object

Define all the models and combinations to try out with their hyperparameters. 

In [5]:
random_state=42

# Continuous Imputer List (list of tuples with unique strings and corresponding instances)
continuous_imputer_list = [
    ("KNNImputer_5", KNNImputer(n_neighbors=5)),
]

# Ordinal Imputer List (list of tuples with unique strings and corresponding instances)
ordinal_imputer_list = [
    ("KNNImputer1", KNNImputer(n_neighbors=1)),
]

# Predictive Models List (list of tuples with unique strings and corresponding instances)
predictive_models_list = [
    ("LinearRegression", LinearRegression()),
    ("MultiTaskElasticNet", MultiTaskElasticNet()),
    ("MultiTaskElasticNet_tuned", MultiTaskElasticNet(**{'alpha': 0.1, 'l1_ratio': 0.1})),
    ("MultiTaskLasso", MultiTaskLasso()),
    ("MultiTaskLasso_tuned", MultiTaskLasso(**{'alpha': 0.001})),
    ("RandomForestRegressor", RandomForestRegressor()),
    ("XGBoostRegressor", XGBoostRegressor()),
    ("XGBoostRegressor_tuned", XGBoostRegressor(**{'colsample_bytree': 0.8776807051588262, 'learning_rate': 0.13329520360246094, 'max_depth': 8, 'min_child_weight': 4, 'subsample': 0.5924272277627636})),
    ("TabNetRegressor_default", TabNetModelWrapper(n_a=8, n_d=8)),
    ("TabNetRegressor_custom", TabNetModelWrapper(n_a=32, n_d=32)),
    ("PLSRegression_4_components", PLSRegression(n_components=4))
]

In [6]:
ordinal_features = ['APOE_epsilon2', 'APOE_epsilon3', 'APOE_epsilon4']
continuous_features = [col for col in df_X_train.columns if col not in ordinal_features]

# Prepare Tabular configurations (shared for all PyTorch models)
data_config = DataConfig(
    target=df_y_train.columns.tolist(),
    continuous_cols=continuous_features,
    categorical_cols=ordinal_features
)
trainer_config = TrainerConfig(
    batch_size=1024, max_epochs=10, auto_lr_find=False,
    early_stopping="valid_loss", early_stopping_mode="min", early_stopping_patience=5,
    checkpoints="valid_loss", load_best=True, progress_bar="nones",
)
optimizer_config = OptimizerConfig()
head_config = LinearHeadConfig(dropout=0.1).__dict__

predictive_models_list += [
    ("GatedAdditiveTreeEnsembleConfig_tab", 
    TabularModelWrapper(
        GatedAdditiveTreeEnsembleConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        gflu_stages=6,
        gflu_dropout=0.0,
        tree_depth=5,
        num_trees=20,
        chain_trees=False,
        share_head_weights=True), data_config, trainer_config, optimizer_config 
    )),
    ("DANetConfig_tab",
    TabularModelWrapper(
        DANetConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_layers=8,
        k=5,
        dropout_rate=0.1), data_config, trainer_config, optimizer_config
    )),
    ("TabTransformerConfig_tab",
        TabularModelWrapper(
        TabTransformerConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        embedding_initialization="kaiming_uniform",
        embedding_bias=False), data_config, trainer_config, optimizer_config
    )),
    ("TabNetModelConfig_tab",
        TabularModelWrapper(
        TabNetModelConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_d=8,
        n_a=8,
        n_steps=3,
        gamma=1.3,
        n_independent=2,
        n_shared=2), data_config, trainer_config, optimizer_config
    )),
]

In [7]:
# Generate all combinations
combinations = list(product(continuous_imputer_list, ordinal_imputer_list, predictive_models_list))

# Display all combinations
for continuous_imputer, ordinal_imputer, model in combinations:
    print(f"Continuous Imputer: {continuous_imputer[0]}, Ordinal Imputer: {ordinal_imputer[0]}, Model: {model[0]}")

print(f"Combinations of preprocessing and models to test : {len(combinations)}")

Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: LinearRegression
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: MultiTaskElasticNet
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: MultiTaskElasticNet_tuned
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: MultiTaskLasso
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: MultiTaskLasso_tuned
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: RandomForestRegressor
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: XGBoostRegressor
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: XGBoostRegressor_tuned
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: TabNetRegressor_default
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: TabNetRegressor_custom
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: PLSRegressi

In [8]:
# Initialize HDF5 file
results_file = '../pickle/training_2_dict_results.pickle'

if os.path.exists(results_file): 

    with open(results_file, "rb") as input_file:
        all_dict_results = pickle.load(input_file)

else : 
    all_dict_results = []

In [9]:
for result in all_dict_results:
    print(result["params"])

In [10]:
if False : 
    params_comb = [{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'GatedAdditiveTreeEnsembleConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'DANetConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'TabTransformerConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'TabNetModelConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'GatedAdditiveTreeEnsembleConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'DANetConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabTransformerConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabNetModelConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]}]

    for params in params_comb:
        all_dict_results = clean_dict_list(all_dict_results, remove_if_none=False, remove_key_val={"params": params})

In [11]:
all_dict_results = clean_dict_list(all_dict_results, remove_if_none=False, remove_key_val={'fitting_time':None})

In [12]:
for continuous_imputer, ordinal_imputer, model in combinations:
    name_continuous_imputer, continuous_imputer_instance = continuous_imputer
    name_ordinal_imputer, ordinal_imputer_instance = ordinal_imputer
    name_model, model_instance = model

    params = {
        "ordinal_imputer": name_ordinal_imputer, 
        "continuous_imputer": name_continuous_imputer, 
        "model": name_model, "train_shape" : df_X_train.shape, 
        "test_shape": df_X_test.shape
    }
    print(f"Training :{name_model}")

    if any(result['params'] == params for result in all_dict_results):
        # Skip this iteration if the combination exists
        print(f"Skipping existing combination: {params.values()}")
        
        continue

    try: 
    
        # Now you can call your `train_model` function with these components
        dict_results = train_imputer_model(
            df_X_train, df_X_test, df_y_train, df_y_test,
            c_train, c_test,
            ordinal_imputer_instance, name_ordinal_imputer,
            continuous_imputer_instance, name_continuous_imputer,
            model_instance, name_model,
            separate_imputers=True  # Or however you want to specify
        )

    except Exception as e:  

        print(e)
    
        dict_results = {
        "params": params, 
        "imputation_time": None,
        "fitting_time": None, 
        "results_adj": None, 
        "results_org": None
    }
        
    # Optionally keep the all_dict_results list updated
    all_dict_results.append(dict_results)

        # Save the updated results back to the pickle file
    with open(results_file, 'wb') as f:
        pickle.dump(all_dict_results, f)


Training :LinearRegression
Using separate imputers for ordinal and continuous data.
No NaN in test data -> Keep as it is. 
Training :MultiTaskElasticNet
Using separate imputers for ordinal and continuous data.
No NaN in test data -> Keep as it is. 
Training :MultiTaskElasticNet_tuned
Using separate imputers for ordinal and continuous data.
No NaN in test data -> Keep as it is. 
Training :MultiTaskLasso
Using separate imputers for ordinal and continuous data.
No NaN in test data -> Keep as it is. 
Training :MultiTaskLasso_tuned
Using separate imputers for ordinal and continuous data.
No NaN in test data -> Keep as it is. 
Training :RandomForestRegressor
Using separate imputers for ordinal and continuous data.
No NaN in test data -> Keep as it is. 
Training :XGBoostRegressor
Using separate imputers for ordinal and continuous data.
No NaN in test data -> Keep as it is. 
Training :XGBoostRegressor_tuned
Using separate imputers for ordinal and continuous data.
No NaN in test data -> Keep as

2025-06-14 09:45:16,442 - {pytorch_tabular.tabular_model:146} - INFO - Experiment Tracking is turned off

Seed set to 42


2025-06-14 09:45:16,457 - {pytorch_tabular.tabular_model:548} - INFO - Preparing the DataLoaders

2025-06-14 09:45:16,461 - {pytorch_tabular.tabular_datamodule:522} - INFO - Setting up the datamodule for          
regression task

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

2025-06-14 09:45:57,281 - {pytorch_tabular.tabular_model:689} - INFO - Training the model completed

2025-06-14 09:45:57,284 - {pytorch_tabular.tabular_model:1529} - INFO - Loading the best model

In [13]:
# Store data (serialize)
with open(results_file, 'wb') as handle:
    pickle.dump(all_dict_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
with open('../pickle/training_2_dict_results.pickle', "rb") as input_file:
    dict_results_split = pickle.load(input_file)

In [15]:
dict_results_split

[{'params': {'ordinal_imputer': 'KNNImputer1',
   'continuous_imputer': 'KNNImputer_5',
   'model': 'LinearRegression',
   'train_shape': (2881, 256),
   'test_shape': (13, 256)},
  'imputation_time': 2.546679735183716,
  'fitting_time': 0.11383223533630371,
  'results_adj': {'mse_score': array([0.90495076, 0.49760788, 0.55481313, 0.90373644]),
   'mae_score': array([0.69934173, 0.54078073, 0.63081682, 0.76542299]),
   'r2': array([ 0.08538623,  0.4353442 , -0.15842057, -0.09932378]),
   'explained_variance': array([ 0.1991749 ,  0.49904245, -0.1551376 ,  0.15684105]),
   'corr': array([0.48281869, 0.70747809, 0.19354395, 0.44611662])},
  'results_org': {'mse_score': array([0.90495075, 0.49760787, 0.55481314, 0.90373643]),
   'mae_score': array([0.69934172, 0.54078073, 0.63081684, 0.76542298]),
   'r2': array([ 0.0373656 ,  0.44254187, -0.08059966, -0.05664115]),
   'explained_variance': array([ 0.1571286 ,  0.50542817, -0.07753724,  0.18957775]),
   'corr': array([0.44167655, 0.712041

# Train models only on MRI features to compare performances

## Test train split

In [16]:
idx_train = list(df_X.isna().any(axis=1))
idx_test = list(~df_X.isna().any(axis=1))

set_intersect_rid = set(df_all[idx_train].RID).intersection(set(df_all[idx_test].RID))
intersect_rid_idx = df_all.RID.isin(set_intersect_rid)

for i, bool_test in enumerate(idx_test): 
    if intersect_rid_idx.iloc[i] & bool_test:
        idx_test[i] = False
        idx_train[i] = True

In [17]:
df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]] = df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]].astype("int", errors='ignore')

In [18]:
df_X_train = df_X[dict_select["MRIth"]].loc[idx_train]
df_X_test = df_X[dict_select["MRIth"]].loc[idx_test]

df_y_train = df_y.loc[idx_train]
df_y_test = df_y.loc[idx_test]

c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

In [19]:
random_state=42
n_imputation_iter = 10

# Continuous Imputer List (list of tuples with unique strings and corresponding instances)
continuous_imputer_list = [
    ("NoImputer", KNNImputer(n_neighbors=1)),

]

# Ordinal Imputer List (list of tuples with unique strings and corresponding instances)
ordinal_imputer_list = [
    ("NoImputer", SimpleImputer(strategy="most_frequent")),
]

# Predictive Models List (list of tuples with unique strings and corresponding instances)
predictive_models_list = [
    ("LinearRegression", LinearRegression()),
    ("MultiTaskElasticNet", MultiTaskElasticNet()),
    ("MultiTaskElasticNet_tuned", MultiTaskElasticNet(**{'alpha': 0.01, 'l1_ratio': 0.01})),
    ("MultiTaskLasso", MultiTaskLasso()),
    ("MultiTaskLasso_tuned", MultiTaskLasso(**{'alpha': 0.001})),
    ("RandomForestRegressor", RandomForestRegressor()),
    ("XGBoostRegressor", XGBoostRegressor()),
    ("XGBoostRegressor_tuned", XGBoostRegressor(**{'colsample_bytree': 0.8776807051588262, 'learning_rate': 0.13329520360246094, 'max_depth': 8, 'min_child_weight': 4, 'subsample': 0.5924272277627636})),
    ("TabNetRegressor_default", TabNetModelWrapper(n_a=8, n_d=8)),
    ("TabNetRegressor_custom", TabNetModelWrapper(n_a=32, n_d=32)),
    ("PLSRegression_4_components", PLSRegression(n_components=4))
]

In [20]:
ordinal_features = ['APOE_epsilon2', 'APOE_epsilon3', 'APOE_epsilon4']
continuous_features = [col for col in df_X_train.columns if col not in ordinal_features]

# Prepare Tabular configurations (shared for all PyTorch models)
data_config = DataConfig(
    target=df_y_train.columns.tolist(),
    continuous_cols=continuous_features,
    categorical_cols=[]
)
trainer_config = TrainerConfig(
    batch_size=1024, max_epochs=10, auto_lr_find=False,
    early_stopping="valid_loss", early_stopping_mode="min", early_stopping_patience=5,
    checkpoints="valid_loss", load_best=True, progress_bar="nones",
)
optimizer_config = OptimizerConfig()
head_config = LinearHeadConfig(dropout=0.1).__dict__

predictive_models_list += [
    ("GatedAdditiveTreeEnsembleConfig_tab", 
    TabularModelWrapper(
        GatedAdditiveTreeEnsembleConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        gflu_stages=6,
        gflu_dropout=0.0,
        tree_depth=5,
        num_trees=20,
        chain_trees=False,
        share_head_weights=True), data_config, trainer_config, optimizer_config 
    )),
    ("DANetConfig_tab",
    TabularModelWrapper(
        DANetConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_layers=8,
        k=5,
        dropout_rate=0.1), data_config, trainer_config, optimizer_config
    )),
    ("TabTransformerConfig_tab",
        TabularModelWrapper(
        TabTransformerConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        embedding_initialization="kaiming_uniform",
        embedding_bias=False), data_config, trainer_config, optimizer_config
    )),
    ("TabNetModelConfig_tab",
        TabularModelWrapper(
        TabNetModelConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_d=8,
        n_a=8,
        n_steps=3,
        gamma=1.3,
        n_independent=2,
        n_shared=2), data_config, trainer_config, optimizer_config
    )),
]

In [21]:
# Generate all combinations
combinations = list(product(continuous_imputer_list, ordinal_imputer_list, predictive_models_list))

# Display all combinations
for continuous_imputer, ordinal_imputer, model in combinations:
    print(f"Continuous Imputer: {continuous_imputer[0]}, Ordinal Imputer: {ordinal_imputer[0]}, Model: {model[0]}")

print(f"Combinations of preprocessing and models to test : {len(combinations)}")

Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: LinearRegression
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskElasticNet
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskElasticNet_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskLasso
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskLasso_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: RandomForestRegressor
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: XGBoostRegressor
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: XGBoostRegressor_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: TabNetRegressor_default
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: TabNetRegressor_custom
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: PLSRegression_4_components
Continuous Imputer: NoImputer, Ordinal 

In [22]:
# Initialize HDF5 file
results_file = '../pickle/training_2_dict_results.pickle'

with open(results_file, "rb") as input_file:
    all_dict_results = pickle.load(input_file)

In [23]:
for continuous_imputer, ordinal_imputer, model in combinations:
    name_continuous_imputer, continuous_imputer_instance = continuous_imputer
    name_ordinal_imputer, ordinal_imputer_instance = ordinal_imputer
    name_model, model_instance = model

    try: 
    
        # Now you can call your `train_model` function with these components
        dict_results = train_imputer_model(
            df_X_train, df_X_test, df_y_train, df_y_test,
            c_train, c_test,
            ordinal_imputer_instance, name_ordinal_imputer,
            continuous_imputer_instance, name_continuous_imputer,
            model_instance, name_model,
            separate_imputers=True  # Or however you want to specify
        )

    except Exception as e:  

        print(e)
    
        params = {
        "ordinal_imputer": name_ordinal_imputer, 
        "continuous_imputer": name_continuous_imputer, 
        "model": name_model, "train_shape" : df_X_train.shape, 
        "test_shape": df_X_test.shape
    }
        dict_results = {
        "params": params, 
        "imputation_time": None,
        "fitting_time": None, 
        "results_adj": None, 
        "results_org": None
    }
        
    # Optionally keep the all_dict_results list updated
    all_dict_results.append(dict_results)

    # Save the updated results back to the pickle file
    with open(results_file, 'wb') as f:
        pickle.dump(all_dict_results, f)

Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Using separate imputers for ordinal and continuo

2025-06-14 09:47:16,212 - {pytorch_tabular.tabular_model:146} - INFO - Experiment Tracking is turned off

Seed set to 42


2025-06-14 09:47:16,224 - {pytorch_tabular.tabular_model:548} - INFO - Preparing the DataLoaders

2025-06-14 09:47:16,227 - {pytorch_tabular.tabular_datamodule:522} - INFO - Setting up the datamodule for          
regression task

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

2025-06-14 09:47:39,387 - {pytorch_tabular.tabular_model:689} - INFO - Training the model completed

2025-06-14 09:47:39,388 - {pytorch_tabular.tabular_model:1529} - INFO - Loading the best model

In [24]:
# Store data (serialize)
with open(results_file, 'wb') as handle:
    pickle.dump(all_dict_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Print Table for reporting

In [25]:
results_file = "../pickle/training_2_dict_results.pickle"

In [26]:
with open(results_file, "rb") as input_file:
    all_dict_results = pickle.load(input_file)

In [27]:
pd.DataFrame(all_dict_results)

,params,imputation_time,fitting_time,results_adj,results_org
0,"{'ordinal_imputer': 'KNNImputer1', 'continuous...",2.546680,0.113832,"{'mse_score': [0.9049507588960866, 0.497607875...","{'mse_score': [0.9049507510973738, 0.497607869..."
1,"{'ordinal_imputer': 'KNNImputer1', 'continuous...",2.489232,0.018780,"{'mse_score': [1.209066965369377, 0.9448829134...","{'mse_score': [1.2090669743425486, 0.944882911..."
2,"{'ordinal_imputer': 'KNNImputer1', 'continuous...",2.492828,1.172759,"{'mse_score': [0.9081049470776361, 0.552868544...","{'mse_score': [0.9081049441441496, 0.552868544..."
3,"{'ordinal_imputer': 'KNNImputer1', 'continuous...",2.539802,0.013393,"{'mse_score': [1.3936376654364884, 1.040563386...","{'mse_score': [1.39363767778841, 1.04056338628..."
4,"{'ordinal_imputer': 'KNNImputer1', 'continuous...",2.532880,1.073404,"{'mse_score': [0.9069250538351172, 0.502013168...","{'mse_score': [0.906925046254779, 0.5020131630..."
5,"{'ordinal_imputer': 'KNNImputer1', 'continuous...",2.543471,39.735521,"{'mse_score': [0.8476480766054094, 0.590657743...","{'mse_score': [0.8476480755917476, 0.590657743..."
6,"{'ordinal_imputer': 'KNNImputer1', 'continuous...",2.513994,1.094394,"{'mse_score': [0.8644967418117477, 0.474979766...","{'mse_score': [0.8644967402358948, 0.474979767..."
7,"{'ordinal_imputer': 'KNNImputer1', 'continuous...",2.542050,4.451472,"{'mse_score': [0.6308271979968313, 0.462175000...","{'mse_score': [0.6308271982625578, 0.462175006..."
8,"{'ordinal_imputer': 'KNNImputer1', 'continuous...",2.494494,13.224293,"{'mse_score': [0.8216751045368837, 0.840482296...","{'mse_score': [0.8216750921763722, 0.840482310..."
9,"{'ordinal_imputer': 'KNNImputer1', 'continuous...",2.566462,14.135967,"{'mse_score': [0.6667009562298352, 0.377261128...","{'mse_score': [0.6667009546426534, 0.377261127..."


In [28]:
def generate_metric_table(
    results_list,
    targets,
    metric_name,
    source="Adjusted",
    float_format="%.3f",
    csv_filename=None,
    sort_order="ascending"
):
    """
    Create a LaTeX and CSV table for a single metric across targets, models, and imputers,
    including mean ± std for performance, imputation time, and fitting time.

    Parameters
    ----------
    results_list : list of dict
        List of experiment results.
    targets : list of str
        Target names (e.g., ['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN']).
    metric_name : str
        Metric to extract (e.g., 'mae_score').
    source : str
        'Adjusted' or 'Original'.
    float_format : str
        Format for floats (e.g., '%.3f').
    csv_filename : str or None
        If provided, saves the table to CSV.
    sort_order : str
        'ascending' or 'descending' for sorting by mean.

    Returns
    -------
    df : pd.DataFrame
        Final formatted DataFrame.
    latex_table : str
        LaTeX-formatted table string.
    """
    rows = []
    version_key = "results_adj" if source.lower() == "adjusted" else "results_org"

    for res in results_list:
        result_block = res.get(version_key)
        if result_block is None:
            continue

        metric_values = result_block.get(metric_name)
        if metric_values is None:
            continue

        if len(metric_values) != len(targets):
            continue

        ordinal_imputer = res["params"].get("ordinal_imputer")
        continuous_imputer = res["params"].get("continuous_imputer")
        model = res["params"].get("model")

        values = np.array(metric_values, dtype=np.float64)
        mean_val = np.mean(values)
        std_val = np.std(values)

        # Time metrics
        imp_times = np.array(res.get("imputation_time", []), dtype=np.float64)
        fit_times = np.array(res.get("fitting_time", []), dtype=np.float64)

        row = {
            "Ordinal Imputer": ordinal_imputer,
            "Continuous Imputer": continuous_imputer,
            "Model": model,
            "Mean": mean_val,
            "Mean ± SD": f"{mean_val:.3f} ± {std_val:.3f}",
            "Imputation Time": f"{imp_times.mean():.2f}" if imp_times.size > 0 else "N/A",
            "Fitting Time": f"{fit_times.mean():.2f}" if fit_times.size > 0 else "N/A"
        }

        row.update({target: val for target, val in zip(targets, values)})
        rows.append(row)

    df = pd.DataFrame(rows)

    # Reorder columns for display
    display_cols = (
        ["Ordinal Imputer", "Continuous Imputer", "Model"] +
        targets +
        ["Mean ± SD", "Imputation Time", "Fitting Time"]
    )
    df = df.sort_values(by="Mean", ascending=(sort_order == "ascending"))
    df = df[display_cols]

    df.drop_duplicates(subset=["Ordinal Imputer", "Continuous Imputer", "Model"] +
        targets +
        ["Mean ± SD",], inplace=True)

    # Save CSV if requested
    if csv_filename:
        df.to_csv(csv_filename, index=False)

    # Generate LaTeX table
    latex_table = df.to_latex(
        index=False,
        escape=False,
        float_format=float_format,
        caption=f"{metric_name.replace('_', ' ').upper()} across targets with timing info",
        label=f"tab:{metric_name}",
        longtable=False
    )

    return df, latex_table


In [29]:
latex_df, latex_mae = generate_metric_table(
    results_list=all_dict_results,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='corr',
    source="Adjusted",
    csv_filename="../tables/2_training_train_test_corr_adjusted_sorted.csv",
    sort_order="descending"
)
print(latex_mae)

\begin{table}
\caption{CORR across targets with timing info}
\label{tab:corr}
\begin{tabular}{lllrrrrlll}
\toprule
Ordinal Imputer & Continuous Imputer & Model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & Mean ± SD & Imputation Time & Fitting Time \\
\midrule
KNNImputer1 & KNNImputer_5 & TabNetRegressor_custom & 0.745 & 0.795 & 0.659 & 0.594 & 0.698 ± 0.077 & 2.57 & 14.14 \\
KNNImputer1 & KNNImputer_5 & TabNetRegressor_default & 0.749 & 0.626 & 0.391 & 0.739 & 0.626 ± 0.144 & 2.49 & 13.22 \\
KNNImputer1 & KNNImputer_5 & RandomForestRegressor & 0.631 & 0.656 & 0.356 & 0.729 & 0.593 ± 0.141 & 2.54 & 39.74 \\
KNNImputer1 & KNNImputer_5 & XGBoostRegressor_tuned & 0.706 & 0.732 & 0.179 & 0.717 & 0.584 ± 0.234 & 2.54 & 4.45 \\
KNNImputer1 & KNNImputer_5 & XGBoostRegressor & 0.592 & 0.752 & 0.300 & 0.654 & 0.575 ± 0.168 & 2.51 & 1.09 \\
KNNImputer1 & KNNImputer_5 & GatedAdditiveTreeEnsembleConfig_tab & 0.684 & 0.574 & 0.147 & 0.767 & 0.543 ± 0.238 & 2.69 & 16.59 \\
NoImputer & NoImputer & Gate

In [30]:
latex_df, latex_mae = generate_metric_table(
    results_list=all_dict_results,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='r2',
    source="Adjusted",
    csv_filename="../tables/2_training_train_test_r2_adjusted_sorted.csv",
    sort_order="descending"
)
print(latex_mae)

\begin{table}
\caption{R2 across targets with timing info}
\label{tab:r2}
\begin{tabular}{lllrrrrlll}
\toprule
Ordinal Imputer & Continuous Imputer & Model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & Mean ± SD & Imputation Time & Fitting Time \\
\midrule
KNNImputer1 & KNNImputer_5 & TabNetRegressor_custom & 0.326 & 0.572 & 0.433 & 0.170 & 0.375 ± 0.147 & 2.57 & 14.14 \\
KNNImputer1 & KNNImputer_5 & XGBoostRegressor_tuned & 0.362 & 0.476 & -0.098 & 0.376 & 0.279 ± 0.222 & 2.54 & 4.45 \\
KNNImputer1 & KNNImputer_5 & XGBoostRegressor & 0.126 & 0.461 & 0.030 & 0.175 & 0.198 ± 0.161 & 2.51 & 1.09 \\
KNNImputer1 & KNNImputer_5 & RandomForestRegressor & 0.143 & 0.330 & 0.111 & 0.184 & 0.192 ± 0.084 & 2.54 & 39.74 \\
KNNImputer1 & KNNImputer_5 & GatedAdditiveTreeEnsembleConfig_tab & 0.275 & 0.324 & -0.004 & 0.116 & 0.178 ± 0.130 & 2.69 & 16.59 \\
NoImputer & NoImputer & XGBoostRegressor & -0.008 & 0.373 & -0.040 & 0.092 & 0.104 ± 0.162 & nan & 1.03 \\
KNNImputer1 & KNNImputer_5 & MultiTaskElas

In [31]:
latex_df, latex_mse = generate_metric_table(
    results_list=all_dict_results,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='mse_score',
    source="Adjusted",
    csv_filename="../tables/2_training_train_test_mse_adjusted_sorted.csv",
    sort_order="ascending"
)
print(latex_mse)

\begin{table}
\caption{MSE SCORE across targets with timing info}
\label{tab:mse_score}
\begin{tabular}{lllrrrrlll}
\toprule
Ordinal Imputer & Continuous Imputer & Model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & Mean ± SD & Imputation Time & Fitting Time \\
\midrule
KNNImputer1 & KNNImputer_5 & TabNetRegressor_custom & 0.667 & 0.377 & 0.271 & 0.682 & 0.499 ± 0.179 & 2.57 & 14.14 \\
KNNImputer1 & KNNImputer_5 & XGBoostRegressor_tuned & 0.631 & 0.462 & 0.526 & 0.513 & 0.533 ± 0.061 & 2.54 & 4.45 \\
KNNImputer1 & KNNImputer_5 & XGBoostRegressor & 0.864 & 0.475 & 0.465 & 0.678 & 0.621 ± 0.165 & 2.51 & 1.09 \\
KNNImputer1 & KNNImputer_5 & GatedAdditiveTreeEnsembleConfig_tab & 0.718 & 0.595 & 0.481 & 0.727 & 0.630 ± 0.101 & 2.69 & 16.59 \\
KNNImputer1 & KNNImputer_5 & RandomForestRegressor & 0.848 & 0.591 & 0.426 & 0.670 & 0.634 ± 0.152 & 2.54 & 39.74 \\
NoImputer & NoImputer & XGBoostRegressor & 0.997 & 0.553 & 0.498 & 0.746 & 0.699 ± 0.196 & nan & 1.03 \\
KNNImputer1 & KNNImputer_5 & Mul

In [32]:
latex_df, latex_mae = generate_metric_table(
    results_list=all_dict_results,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='mae_score',
    source="Adjusted",
    csv_filename="../tables/2_training_train_test_mae_adjusted_sorted.csv",
    sort_order="ascending"
)
print(latex_mae)

\begin{table}
\caption{MAE SCORE across targets with timing info}
\label{tab:mae_score}
\begin{tabular}{lllrrrrlll}
\toprule
Ordinal Imputer & Continuous Imputer & Model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & Mean ± SD & Imputation Time & Fitting Time \\
\midrule
KNNImputer1 & KNNImputer_5 & TabNetRegressor_custom & 0.608 & 0.518 & 0.410 & 0.668 & 0.551 ± 0.097 & 2.57 & 14.14 \\
KNNImputer1 & KNNImputer_5 & XGBoostRegressor_tuned & 0.599 & 0.588 & 0.655 & 0.562 & 0.601 ± 0.034 & 2.54 & 4.45 \\
KNNImputer1 & KNNImputer_5 & XGBoostRegressor & 0.675 & 0.589 & 0.627 & 0.628 & 0.630 ± 0.031 & 2.51 & 1.09 \\
KNNImputer1 & KNNImputer_5 & RandomForestRegressor & 0.671 & 0.648 & 0.617 & 0.598 & 0.634 ± 0.028 & 2.54 & 39.74 \\
NoImputer & NoImputer & XGBoostRegressor_tuned & 0.801 & 0.516 & 0.658 & 0.626 & 0.650 ± 0.102 & nan & 5.83 \\
KNNImputer1 & KNNImputer_5 & GatedAdditiveTreeEnsembleConfig_tab & 0.623 & 0.679 & 0.641 & 0.663 & 0.652 ± 0.021 & 2.69 & 16.59 \\
KNNImputer1 & KNNImputer_5

In [33]:
latex_df.Model.value_counts()

Model
TabNetRegressor_custom                 2
XGBoostRegressor_tuned                 2
XGBoostRegressor                       2
RandomForestRegressor                  2
GatedAdditiveTreeEnsembleConfig_tab    2
LinearRegression                       2
MultiTaskLasso_tuned                   2
MultiTaskElasticNet_tuned              2
TabNetRegressor_default                2
PLSRegression_4_components             2
DANetConfig_tab                        2
MultiTaskElasticNet                    2
MultiTaskLasso                         2
TabNetModelConfig_tab                  2
TabTransformerConfig_tab               2
Name: count, dtype: int64